In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('cleaned_data.csv')

In [3]:
df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [4]:
selected_features = ["HighBP", "Diabetes_binary", "HighChol", "Stroke",
                     "GenHlth", "Age", "DiffWalk", "PhysHlth", "HeartDiseaseorAttack","Smoker","Income"]
df_selected = df[selected_features]
data=df_selected

In [5]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X = data.drop('HeartDiseaseorAttack', axis=1)
y = data['HeartDiseaseorAttack']
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("X_train_resampled shape:", X_resampled.shape)
print("y_train_resampled shape:", y_resampled.shape)


X_train_resampled shape: (411522, 10)
y_train_resampled shape: (411522,)


In [6]:
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42)


In [7]:
pip install xgboost


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [10]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "LogisticRegression": LogisticRegression( random_state=42),
    "PolynomialPipeline": Pipeline([
        ("poly", PolynomialFeatures(degree=2)),
        ("clf", LogisticRegression(random_state=42))
    ]),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

mlflow.set_experiment("HeartDisease")

for name, model in models.items():
    with mlflow.start_run(run_name=name):
        model.fit(X_train_resampled, y_train_resampled)
        y_pred = model.predict(X_test_resampled)

        
        report_dict = classification_report(y_test_resampled, y_pred, output_dict=True)

        for label, metrics in report_dict.items():
            if isinstance(metrics, dict):  
                for metric_name, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric_name}", value)
            elif label == "accuracy":
                mlflow.log_metric("accuracy", metrics)

        mlflow.sklearn.log_model(model, name)

        print(f"Logged {name} run to MLflow.")


2025/05/08 21:51:06 INFO mlflow.tracking.fluent: Experiment with name 'HeartDisease' does not exist. Creating a new experiment.
2025/05/08 21:51:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged RandomForest run to MLflow.
🏃 View run RandomForest at: http://127.0.0.1:5000/#/experiments/458964212107376866/runs/7eea927e7b7343eebd6803d74fdb387a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/458964212107376866


2025/05/08 21:51:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged LogisticRegression run to MLflow.
🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/458964212107376866/runs/dd929025ca324becbe5f3701346147dc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/458964212107376866


/home/yassin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/05/08 21:51:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/home/yassin/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:51:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Logged PolynomialPipeline run to MLflow.
🏃 View run PolynomialPipeline at: http://127.0.0.1:5000/#/experiments/458964212107376866/runs/2cf1dfcf0cc344a3ae88d60241ba9243
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/458964212107376866


2025/05/08 21:51:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged XGBoost run to MLflow.
🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/458964212107376866/runs/b69ee7620be349efa24b47b25fcf8c06
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/458964212107376866
